# Model (Train & Test) **TFIDF**
### By **Néstor Suat** in 2019

**Descripción:** Entrenando y probando SVM con TFIDF. 

**Input:**
* Train and Test set
* Parameters TFIDF

**Output:**
* Metrics: confusion matrix, accuracy, recall, precision and F1-score

***

## 0. Cargando datos y limpieza

### Importando librerías

Como estamos en un archivo afuera se necesita agregar la dirección ../ (raíz del proyexto) para importar la librería de preprocesamiento.

In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, '../../../')

from classes.tfidf.preprocessing import Preprocessing as tfidf

In [2]:
def vector_to_phrase(vector, tfidf):
    c = 0
    sentence = []
    for i in vector:
        if i!=0.0:        
            sentence.append(tfidf.get_feature_names()[c])
        c+=1
    phrase = " ".join(sentence)
    return phrase

### Importando datasets

In [3]:
train = pd.read_csv("../../../data/v1/7030/train70.tsv", delimiter = "\t", quoting = 3)
test = pd.read_csv("../../../data/v1/7030/test30.tsv", delimiter = "\t", quoting = 3)
dataset = pd.concat([train,test])
print(train.shape, test.shape) # (3804, 3)

(2662, 2) (1142, 2)


### Preprocessing

In [4]:
type_clean = 4 #Tiene que ser el mismo que 'file' (prefijo)

#TFIDF
max_df = 0.5    
max_features = None
min_df = 0.001    
ngram_range = (1, 2)

#Model SVM
C=4
gamma=0.4
kernel= 'rbf'

Se hace la limpieza de los dos conjuntos de datos (prueba y entrenamiento)

In [5]:
clean = tfidf(train)
clean.fit_clean(type_clean)
train.head()

,text,label,clean
0,📢#Atención: se presenta siniestro vial entre u...,1,atención se presenta siniestro vial entre...
1,📢#Atención: a esta hora se presentan disturbio...,0,atención a esta hora se presentan disturb...
2,Incidente vial entre taxi 🚖 y‍ motocicleta 🏍️ ...,1,incidente vial entre taxi y motocicleta ...
3,@chemabernal @Moniva0517 @MartinSantosR La grá...,0,la gráfica dice que la deuda lp como ...
4,RT @CaracolRadio: #CaracolEsMás | ¡Atención! F...,1,rt caracol esmás atención fuerte ac...


In [6]:
clean_test = tfidf(test)
clean_test.fit_clean(type_clean)
test.head()

,text,label,clean
0,¿Cómo se encuentra el tráfico en la ciudad? 🔴 ...,0,cómo se encuentra el tráfico en la ciudad ...
1,RT @GuavioNoticias: 🚨En horas de la madrugada ...,1,rt en horas de la madrugada se presenta u...
2,"Incidente vial entre moto 🏍️ y taxi 🚕, en la ...",1,incidente vial entre moto y taxi en ...
3,Los supervisores de Asobel prestaron apoyo en ...,1,los supervisores de asobel prestaron apoyo en ...
4,Paso a un carril en la vía Bogotá-Villavicenci...,1,paso a un carril en la vía bogotá villavicenci...


Algunos tweets despúes de la limpieza quedan vacios dado que en el proceso de limpieza se eliminan los tweets con menos de 3 tokens. Lo siguiente es evitar errores para eliminar los tweets que despúes de limpios esten nulos

In [7]:
train = train[~train['clean'].isnull()] #Elimina publicaciones que estan null al eliminarlo porque no generan valor en el proceso de limpieza
test = test[~test['clean'].isnull()]
print(train.shape, test.shape) # (3804, 3)

(2662, 3) (1142, 3)


Se realiza la vectorización del texto

In [8]:
embedding, vectorizer = clean.feature_extraction(ngram_range=ngram_range, max_df=max_df, min_df=min_df, max_features=max_features)

### Feature vector To Text
Este fragmento de abajo es solo para comparar dos salidas distintas según el tipo de limpieza aplicada

### Train & Test set

In [9]:
X_train = embedding[:,1:]
X_train=X_train.astype('float')

y_train = embedding[:,0]
y_train=y_train.astype('int')

El modelo TFIDF se entrena solo con los datos de entrenamiento, asi que para generar el vector de características para el conjunto de prueba se debe hacer lo siguiente.

In [10]:
X_test, y_test = vectorizer.transform(test.clean).toarray(), test.label

## 1. Model

### Support Vector Machine

In [11]:
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import StratifiedKFold
from sklearn import model_selection

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

### Support Vector Machine (**SVM**) Model

### Naive Bayes (**NB**) Model

### Random Fiorest (**RF**) Model

In [12]:
classifier = RandomForestClassifier(n_estimators=100,random_state=100,n_jobs=-1,max_depth=40)

### Evaluación con Cross Validation

Cross validation solo los datos **de validación y entrenamiento**

Cross validation solo **todos los datos**

In [13]:
#dataset = pd.concat([train,test])
clean_all = tfidf(dataset)
clean_all.fit_clean(type_clean)
embedding_all, vectorizer_all = clean_all.feature_extraction(ngram_range=ngram_range, max_df=max_df, min_df=min_df, max_features=max_features)
X_all = embedding_all[:,1:]
X_all=X_all.astype('float')

y_all = embedding_all[:,0]
y_all=y_all.astype('int')

In [14]:
skfold = StratifiedKFold(n_splits=10, random_state=100)

scores = model_selection.cross_val_score(classifier, X_all, y_all, cv=skfold, n_jobs=-1)
print("Accuracy: %0.6f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

scores = model_selection.cross_val_score(classifier, X_all, y_all, cv=skfold, scoring='f1_macro', n_jobs=-1)
print("F1-score: %0.6f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

scores = model_selection.cross_val_score(classifier, X_all, y_all, cv=skfold, scoring='recall_macro', n_jobs=-1)
print("Recall: %0.6f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

scores = model_selection.cross_val_score(classifier, X_all, y_all, cv=skfold, scoring='precision_macro', n_jobs=-1)
print("Precision: %0.6f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.951368 (+/- 0.03)
F1-score: 0.951339 (+/- 0.03)
Recall: 0.951368 (+/- 0.03)
Precision: 0.952321 (+/- 0.02)


### Evaluación con **Train/test**

In [15]:
# Predicting the Test set results
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
cm_svm = confusion_matrix(y_test, y_pred)

metrics_svm = []
metrics = {}
metrics['accuracy'] = accuracy_score(y_test, y_pred)
metrics['recall'] = recall_score(y_test, y_pred)
metrics['precision'] = precision_score(y_test, y_pred)
metrics['f1'] = f1_score(y_test, y_pred)
metrics_svm.append(metrics)
metrics_svm = pd.DataFrame(metrics_svm)

In [16]:
print(metrics_svm)
print(cm_svm)

   accuracy    recall  precision        f1
0  0.949212  0.925267   0.970149  0.947177
[[564  16]
 [ 42 520]]
